In [ ]:
import os

# ... (Your existing imports)

model_path = '/home/agrotech/deep-gauge-master/Reading_Circular_Analogue_Gauges_using_Digital_Image_Processing/model_weights/pose_gauge.pt'
img_folder = '/path/to/your/images/folder'  # Update this to the path of your image folder
output_excel_path = 'angles_results.xlsx'

# Initialize a global DataFrame to store results
all_results_df = pd.DataFrame(columns=['File', 'Angle_ABC', 'Angle_BDC'])

def compute_angle(A, B, C):
    # Your existing compute_angle function
    # ...

def calculate_pointer_angle(img):
    model1 = YOLO(model_path)
    results = model1(img)
    theta1 = None  # Initialize with a default value
    theta2 = None  # Initialize with a default value

    for result in results:
        keypoints_set = result.keypoints.xy.cpu().numpy()

        # Check if keypoints_set is empty
        if keypoints_set.size == 0:
            print('No Key Point Detected')
            theta1 = 0
            theta2 = 45
            break

        # Check if there are at least 4 keypoints in the set
        if keypoints_set.shape[1] < 4:
            print('Not enough keypoints detected')
            break

        A, B, C, D = keypoints_set[0][0], keypoints_set[0][1], keypoints_set[0][3], keypoints_set[0][2]

        theta1 = compute_angle(A, B, C)
        theta2 = 360 - compute_angle(B, D, C)

        print(f"Angle between keypoints A, B, and C is: {theta1:.2f} degrees")
        print(f"Angle between keypoints B, C, and D is: {theta2:.2f} degrees")

    return theta1, theta2

def save_to_excel(file_name, angle_ABC, angle_BDC):
    global all_results_df
    data = {'File': [file_name], 'Angle_ABC': [angle_ABC], 'Angle_BDC': [angle_BDC]}
    df = pd.DataFrame(data)
    
    # Append to the global DataFrame
    all_results_df = all_results_df.append(df, ignore_index=True)

    # Save to Excel
    all_results_df.to_excel(output_excel_path, index=False)

if __name__ == "__main__":
    for file_name in os.listdir(img_folder):
        if file_name.endswith(('.jpg', '.png', '.jpeg')):  # Add other image formats if needed
            img_path = os.path.join(img_folder, file_name)
            image_output = calculate_pointer_angle(img_path)
            save_to_excel(file_name, *image_output)
